# Predict Water Amount Needed

## Get and Prepare Weather Data from DataBase

In [73]:
import pymysql.cursors
import pandas as pd
import numpy as np

pd.options.display.max_rows = 200
#pd.set_option('display.float_format', lambda x: '%.20f' % x) #Display as Float
pd.set_option('display.float_format', lambda x: '{:,}'.format(x)) #Display as Scientific


connection = pymysql.connect(host = "mikmak.cc", user="sensor", passwd="Gaffe2017", db="weatherDW")
query = ('SELECT * FROM log_v_last24Hours WHERE S_Text <> "None"')

with connection.cursor() as cursor:
    cursor.execute(query)
connection.commit()
e_Log = cursor.fetchall()
connection.close()

e_Log = (np.array(e_Log))

138

In [74]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

t_Log = e_Log

t_Log = pd.DataFrame(data = t_Log[1:,:],
                     index = t_Log[1:,2],
                     columns =["ID","Value","TimeStamp"] )

t_Log = t_Log[t_Log != '-'] #DataSet contains some missing values, remove them

t_Log = t_Log.pivot(index='TimeStamp', columns='ID', values='Value')
t_Log = t_Log.sort_index(ascending = True) #It should be ascending, for rolling calculation later
t_Log = t_Log.dropna(how = "any")
t_Log = t_Log.astype(float)

t_Log.tail()


ID,Humidity,Pressure,Rain,Sunshine,Temperature,Wind
TimeStamp,,,,,,
2017-04-25 14:35:48,78.0,951.8,0.4,0.0,11.2,24.8
2017-04-25 15:35:50,85.0,952.1,0.1,0.0,10.4,14.8
2017-04-25 16:35:50,88.0,951.9,0.1,0.0,10.1,5.4
2017-04-25 17:35:50,88.0,951.7,0.1,0.0,9.4,3.6
2017-04-25 18:35:51,88.0,951.6,0.3,0.0,8.9,3.2


## Transform Data into Input Vector

### Order Data and Calculate Means and Sums

In [75]:
X = pd.DataFrame()

lastRow = len(t_Log.axes[0]) - 1

X = X.assign(tre200b0=t_Log.Temperature[[lastRow]])
X = X.assign(ure200b0=t_Log.Humidity[[lastRow]])
X = X.assign(rre200b0=t_Log.Rain[[lastRow]])
X = X.assign(sre000b0=t_Log.Sunshine[[lastRow]])
X = X.assign(fu3010b0=t_Log.Wind[[lastRow]])
X = X.assign(prestab0=t_Log.Pressure[[lastRow]])
#"tre200b0","ure200b0","rre150b0","sre000b0","fu3010b0","prestab0"

t_LogRolling = t_Log.rolling(len(t_Log.axes[0]))

X = X.assign(tre200b0_mean=t_LogRolling.Temperature.mean())
X = X.assign(ure200b0_mean=t_LogRolling.Humidity.mean())
X = X.assign(rre200b0_sum=t_LogRolling.Rain.sum())
X = X.assign(sre000b0_sum=t_LogRolling.Sunshine.sum())
X = X.assign(fu3010b0_mean=t_LogRolling.Wind.mean())
X = X.assign(prestab0_mean=t_LogRolling.Pressure.mean())

X.transpose()

TimeStamp,2017-04-25 18:35:51
tre200b0,8.9
ure200b0,88.0
rre200b0,0.3
sre000b0,0.0
fu3010b0,3.2
prestab0,951.6
tre200b0_mean,12.286363636363633
ure200b0_mean,67.18181818181819
rre200b0_sum,1.2
sre000b0_sum,5.0


### Poly Features etc.

In [76]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from sklearn.preprocessing import PolynomialFeatures

polyDegree = 5
poly = PolynomialFeatures(degree=polyDegree)
X = poly.fit_transform(X).astype(int)

#poly.n_output_features_ 
#pd.DataFrame(X).transpose()

## Classify!
Use Logistic Regression Modell to decide if 
- no water at all or 
- a lot of water or 
- normal amount of water
is needed

### Load Model

In [77]:
from sklearn.externals import joblib
model = joblib.load('data/logreg_5deg.pkl')

### Do Calculation and Show Result
- 0 - means a lot of water
- 1 - is medium
- 2 - device should stay off

In [78]:
result = model.predict(X)[0]
result

from sklearn import preprocessing
le = joblib.load('data/leModel.pkl')
le.inverse_transform(result)


1

'Med'

This is very good!

In [79]:
#pd.DataFrame(model.coef_.transpose())

In [80]:
if result == 1:
    model_reg = joblib.load('data/linreg_med_5deg.pkl')
    res = model_reg.predict(X)[0]
else:
    model_reg = joblib.load('data/linreg_med_5deg.pkl')
    res = model_reg.predict(X)[0]

    
res

369.72291740060382

In [81]:

K = 10
np.argpartition(model_reg.coef_,-K)[-K:]

array([3980, 4176, 4171, 4173, 3998, 4052, 4050, 5748, 5730, 6167])